In [2]:
import os

import openai
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
openai.api_key = os.environ["OPENAI_API_KEY"]


client = OpenAI()

file = client.files.create(
    file=open("parking_violation.csv", "rb"), purpose="assistants"
)

assistant = client.beta.assistants.create(
    name="Data visualizer",
    description="You are great at creating beautiful data visualizations. You analyze data present in .csv files, understand trends, and come up with data visualizations relevant to those trends. You also share a brief text summary of the trends observed.",
    model="gpt-4o",
    tools=[{"type": "code_interpreter"}],
    tool_resources={"code_interpreter": {"file_ids": [file.id]}},
)

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Create 3 data visualizations based on the trends in this file.",
            "attachments": [
                {"file_id": file.id, "tools": [{"type": "code_interpreter"}]}
            ],
        }
    ]
)

# Submit the thread to the assistant (as a new run).
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)
print(f"👉 Run Created: {run.id}")


In [8]:
import time

# Wait for run to complete.
while run.status != "completed":
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(f"🏃 Run Status: {run.status}")
    time.sleep(1)
else:
    print(f"🏁 Run Completed!")

# Get the latest message from the thread.
message_response = client.beta.threads.messages.list(thread_id=thread.id)
messages = message_response.data


🏁 Run Completed!


In [12]:
# Print the latest message.
latest_message = messages[0]
print(latest_message)
# print(f"💬 Response: {latest_message.content[0].text.value}")

Message(id='msg_4eAG4WDmko95o5K2jR5HvEjH', assistant_id='asst_PVcQX7FjZVjaFF2NHJsfTy31', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-yxDf3NDSodEzxxzooqUa9J2a', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='The third visualization shows the top 10 most common violation codes. Violation code 98 appears most frequently, followed by codes 40 and 14.\n\nTo summarize, we visualized:\n1. The trend of violations over time, showing a spike around 2019.\n2. The distribution of vehicle makes with the most violations, where Honda, Nissan, and Toyota dominate.\n3. The distribution of the top 10 violation codes, with code 98 being the most common.\n\nIf you need further analysis or more specific insights, please let me know!'), type='text')], created_at=1718007592, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_i9fsKMwEiX5YvZFRLmVS8l2H', sta